In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
app = dash.Dash(__name__) #initalize dash app
app.title = "IPL Dashboard" #Assigning title to be displayed on tab

In [3]:
#Reading the dataset
df = pd.read_csv("IPL Ball-by-Ball 2008-2020.csv")
df.tail(5)

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
193463,1237181,1,12,5,RR Pant,SS Iyer,NM Coulter-Nile,0,0,0,0,0,NaN,NaN,NaN,NaN,Delhi Capitals,Mumbai Indians
193464,1237181,1,12,6,RR Pant,SS Iyer,NM Coulter-Nile,1,0,1,0,0,NaN,NaN,NaN,NaN,Delhi Capitals,Mumbai Indians
193465,1237181,1,13,1,RR Pant,SS Iyer,KH Pandya,0,1,1,0,0,NaN,NaN,NaN,wides,Delhi Capitals,Mumbai Indians
193466,1237181,1,13,2,RR Pant,SS Iyer,KH Pandya,1,0,1,0,0,NaN,NaN,NaN,NaN,Delhi Capitals,Mumbai Indians
193467,1237181,1,13,3,SS Iyer,RR Pant,KH Pandya,1,0,1,0,0,NaN,NaN,NaN,NaN,Delhi Capitals,Mumbai Indians


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193468 entries, 0 to 193467
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                193468 non-null  int64 
 1   inning            193468 non-null  int64 
 2   over              193468 non-null  int64 
 3   ball              193468 non-null  int64 
 4   batsman           193468 non-null  object
 5   non_striker       193468 non-null  object
 6   bowler            193468 non-null  object
 7   batsman_runs      193468 non-null  int64 
 8   extra_runs        193468 non-null  int64 
 9   total_runs        193468 non-null  int64 
 10  non_boundary      193468 non-null  int64 
 11  is_wicket         193468 non-null  int64 
 12  dismissal_kind    9495 non-null    object
 13  player_dismissed  9495 non-null    object
 14  fielder           6784 non-null    object
 15  extras_type       10233 non-null   object
 16  batting_team      193468 non-null  obj

In [4]:
# Discover team names
df["batting_team"].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Kings XI Punjab', 'Chennai Super Kings', 'Rajasthan Royals',
       'Delhi Daredevils', 'Mumbai Indians', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals'], dtype=object)

In [5]:
# Discover team names
df["bowling_team"].unique()

array(['Royal Challengers Bangalore', 'Kolkata Knight Riders',
       'Chennai Super Kings', 'Kings XI Punjab', 'Delhi Daredevils',
       'Rajasthan Royals', 'Mumbai Indians', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', nan,
       'Sunrisers Hyderabad', 'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals'], dtype=object)

In [6]:
#cleaning the data
team_names = {
                "Sunrisers Hyderabad":"Deccan Chargers",
                "Rising Pune Supergiant": "Rising Pune Supergiants",
                "Delhi Capitals": "Delhi Daredevils"
             }
df["batting_team"] = df["batting_team"].replace(team_names)
df["bowling_team"] = df["bowling_team"].replace(team_names)

In [7]:
df["batting_team"].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Kings XI Punjab', 'Chennai Super Kings', 'Rajasthan Royals',
       'Delhi Daredevils', 'Mumbai Indians', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Gujarat Lions'], dtype=object)

In [8]:
df["is_wicket"].unique()

array([0, 1], dtype=int64)

In [9]:
df.dismissal_kind.unique()

array([nan, 'caught', 'run out', 'bowled', 'lbw', 'retired hurt',
       'stumped', 'caught and bowled', 'hit wicket',
       'obstructing the field'], dtype=object)

In [10]:
#function which will render bar graph for run scored by player
def get_total_runs(player):
    global df
    bat_df = df.groupby(["bowling_team", "batsman"])["batsman_runs"].sum().reset_index().copy()
    bat_df = bat_df[bat_df["batsman"]==player]
    bat_df = bat_df[bat_df["batsman_runs"]>0]

    fig = px.bar(bat_df, x="bowling_team", y="batsman_runs", color="batsman_runs",
             labels={'bowling_team':'Teams', 'batsman_runs':'Runs Scored'}, height=400, width = 500,
                title="Runs Scored by " + player + " against all teams", template='plotly_dark')
    fig.update_layout({'paper_bgcolor': '#282828', 'plot_bgcolor':'#282828'})
    return fig

#function which will render bar graph for wickets by player
def get_total_wickets(player):
    global df
    bol_df = df[~df["dismissal_kind"].isin(["retired hurt", "run out", "obstructing the field"])].copy()
    bol_df = bol_df.groupby(["batting_team", "bowler"])['is_wicket'].sum().reset_index().copy()
    bol_df = bol_df[bol_df["bowler"]==player]
    bol_df = bol_df[bol_df['is_wicket']>0]
                    
    fig = px.pie(bol_df, values='is_wicket', names='batting_team', 
                 labels={'batting_team':'Teams', 'is_wicket':'Wickets_taken'}, height=400, width = 500,
                title="Wickets taken by " + player + " against all teams", template='plotly_dark')
    fig.update_layout({'paper_bgcolor': '#282828', 'plot_bgcolor':'#282828'})
    fig.update_traces(textinfo='value')
    return fig

In [11]:
#creating css for main div 
main_div_style = {"background-color": "#181818", 
                    "padding":"0", 
                    "width":"100%", 
                    "height":"100", 
                    "position": "fixed",
                    "top": "0%",
                    "left": "0",
                    "bottom": "0",
                }

In [12]:
#creating options for dropdown display
options = [{"label":i, "value":i} for i in df["batsman"].unique()] 

#creating dropdown section
dropdown_box = html.Div(children = [
            dcc.Dropdown(
                    id='player_name',              #unique identifier for this element
                    options=options,               #list of players
                    clearable=False,               #this feature helps in not sending an empty value
                    value = "V Kohli",             #setting a default vali
                    placeholder="Select a player", #not really required but just for fun
             )
            ],
            style = {'width': '90%', "position": "fixed", "left": "5%",
                     'display': 'inline-block', "top": "1%", "z-index":"1"}
)

In [13]:
#creating graphs section
Graphs = html.Div(children = [html.Div(children = [
                            dcc.Graph(id = "runsScored")       #graph only requireds unique id in our case
                                    ],
                            style = {'width': '45%', "position": "fixed", "left": "2%",
                                     'display': 'inline-block', "background-color": "#282828",
                                     "top": "10%",
                                    }
                                    ),

                        html.Div(children = [
                                dcc.Graph(id = "wicketsTaken") #graph only requireds unique id in our case
                                ],
                        style = {'width': '45%', "position": "fixed", "left": "53%",
                                 'display': 'inline-block', "background-color": "#282828",
                                 "top": "10%"
                                })
                        ]
)

In [ ]:
#adding the layout to the dash app
app.layout = html.Div(id = "main_div", children =[dropdown_box, Graphs],
                      style = main_div_style)

@app.callback([Output(component_id = "runsScored", component_property = "figure"),
               Output(component_id = "wicketsTaken", component_property = "figure")],
               [Input(component_id = "player_name", component_property = "value")]
             )
def the_callback_function(player_name):
    fig1 = get_total_runs(player_name)
    fig2 = get_total_wickets(player_name)
    return fig1, fig2

if __name__ == "__main__":
    app.run_server(debug=False, port = 8080)


Dash is running on http://127.0.0.1:8080/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Nov/2021 09:20:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 09:20:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 09:20:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 09:20:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 09:20:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 09:20:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 09:20:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 09:20:23] "POST /_dash-update-component HTTP/1.1" 200 -
